|        |         |                                @ |
|:-------|:--------|---------------------------------:|
| Luca   | Mosetti | luca.mosetti-1@studenti.unitn.it |
| Shandy | Darma   |   shandy.darma@studenti.unitn.it |

In [ ]:
from dataclasses import dataclass
from enum import IntEnum, auto
from typing import Iterator, SupportsFloat
from matplotlib_inline.backend_inline import set_matplotlib_formats
from numpy.typing import NDArray

import doctest
import math
import heapq as hq
import itertools as it
import statistics as st
import matplotlib.pyplot as plt
import more_itertools as mit
import numpy as np
import scipy as sp

In [ ]:
set_matplotlib_formats('svg')

# Exercise 1

Implement a discrete-event simulator for an M/M/1 queue-server system that manages at least the stated below:

- Start of the simulation
- End of the simulation
- Arrival of a packet
- Departure of a packet

To do this, create first an ordered queue / list of events where:

- Every event always links to the one that immediately follows it in time;
- When you insert an event in the queue, you always insert it in order of increasing time; (i.e., say that the queue contains three events: event 1 taking place at time $t_1$ and linking to event 2, which takes place at time $t_2$ and which links to event 3 at time $t_3$; if another event 4 taking place at time $t_4$ is inserted in the queue, and $t_2$ < $t_4$ < $t_3$, then you have to make event 2 link to event 4, and event 4 link to event 3.

Finally, implement the system behavior as seen in class, namely:

- When a packet arrives: if the server is free seize the server and schedule the departure of the packet; if the server is busy, increase the number of packets in queue;
- When a departure event is triggered: if the queue is empty, release the server; otherwise keep the server busy and schedule the next departure event.

Use your simulator to do the following:

1. Show how the number of packets in the system (those in queue plus those currently in service) varies over time. Compare your results with the theoretical average number of packets in the system in stationary conditions, $\frac \rho {1 − \rho}$, where $\rho = \frac \lambda \mu$.

2. Play with $\lambda$ and $\mu$, and discuss how their values impact the convergence of the system to the theoretical value.

3. Use your simulator to measure the average time that a packet has to wait in queue, on average, and compare it against the theoretical value, $\frac {\rho^2} {\lambda(1 − \rho)}$ (_Hint_: you will need to run your simulator several times to do the above. Remember the contents of the class on output analysis.)

In [ ]:
class ET(IntEnum):
    START = auto()
    ARRIVAL = auto()
    DEPARTURE = auto()


class LT(IntEnum):
    ARRIVAL = auto()
    SERVING = auto()
    DEPARTURE = auto()


@dataclass(order=True, frozen=True, slots=True)
class Event:
    timestamp: float
    event: ET


@dataclass(order=True, frozen=True, slots=True)
class Log:
    timestamp: float
    log: LT

In [ ]:
seeds: Iterator[int] = mit.sieve(1_000)
mit.consume(seeds, 1)

$$
U \sim \text{Uniform}(0, 1) \qquad X = - \frac {\log U} \lambda \sim \text{Exp}(\lambda)
$$

In [ ]:
def uni_to_exp(lmbd: float, u: float) -> float:
    return -math.log(u) / lmbd


def mm1_simulation(seed_arr: int, seed_dep: int, lmbd: float, mu: float) -> Iterator[Log]:
    """
    Reproducible simulation of M/M/1 queue-server system

    >>> list(mit.take(100, mm1_simulation(3, 7, 1, 1))) == list(mit.take(100, mm1_simulation(3, 7, 1, 1)))
    True

    >>> all(a.timestamp <= b.timestamp for a, b in mit.take(100, it.pairwise(mm1_simulation(3, 7, 1, 1))))
    True

    >>> l1m1 = list(mit.take(100, mm1_simulation(3, 7, 1, 1)))
    >>> mit.quantify(l.log == LT.SERVING for l in l1m1) <= mit.quantify(l.log == LT.ARRIVAL for l in l1m1)
    True

    >>> l1m1 = list(mit.take(100, mm1_simulation(3, 7, 1, 1)))
    >>> mit.quantify(l.log == LT.DEPARTURE for l in l1m1) <= mit.quantify(l.log == LT.SERVING for l in l1m1)
    True

    >>> l1m1 = list(mit.take(100, mm1_simulation(3, 7, 1, 1)))
    >>> l1m2 = list(mit.take(100, mm1_simulation(3, 7, 1, 2)))
    >>> l1m1_dep_over_arr = mit.quantify(l.log == LT.DEPARTURE for l in l1m1) / mit.quantify(l.log == LT.ARRIVAL for l in l1m1)
    >>> l1m2_dep_over_arr = mit.quantify(l.log == LT.DEPARTURE for l in l1m2) / mit.quantify(l.log == LT.ARRIVAL for l in l1m2)
    >>> l1m1_dep_over_arr < l1m2_dep_over_arr
    True
    """

    rng_arr: np.random.Generator = np.random.default_rng(seed_arr)

    def next_arr(timestamp: float) -> Event:
        return Event(
            timestamp + uni_to_exp(lmbd, rng_arr.random()),
            ET.ARRIVAL
        )

    rng_dep: np.random.Generator = np.random.default_rng(seed_dep)

    def next_dep(timestamp: float) -> Event:
        return Event(
            timestamp + uni_to_exp(mu, rng_dep.random()),
            ET.DEPARTURE
        )

    busy: bool = False
    in_queue: int = 0

    timeline: list[Event] = [
        Event(timestamp=0, event=ET.START),
    ]

    while True:
        e: Event = hq.heappop(timeline)
        match e.event:

            case ET.START:
                hq.heappush(timeline, next_arr(e.timestamp))

            case ET.ARRIVAL:
                yield Log(e.timestamp, LT.ARRIVAL)
                hq.heappush(timeline, next_arr(e.timestamp))

                match busy:
                    case True:
                        in_queue = in_queue + 1
                    case False:
                        yield Log(e.timestamp, LT.SERVING)
                        busy = True
                        hq.heappush(timeline, next_dep(e.timestamp))

            case ET.DEPARTURE:
                yield Log(e.timestamp, LT.DEPARTURE)

                match in_queue:
                    case 0:
                        busy = False
                    case _:
                        yield Log(e.timestamp, LT.SERVING)
                        in_queue = in_queue - 1
                        hq.heappush(timeline, next_dep(e.timestamp))


def timespan_packages(xs: Iterator[Log]) -> Iterator[tuple[float, int]]:
    """
    From sequence of logs to sequence of (timespan, packages in system)

    >>> list(timespan_packages([Log(10, LT.ARRIVAL), Log(15, LT.ARRIVAL), Log(25, LT.DEPARTURE), Log(30, LT.DEPARTURE)]))
    [(10, 0), (5, 1), (10, 2), (5, 1)]
    """

    def packages(acc: int, l: LT.ARRIVAL | LT.DEPARTURE) -> int:
        match l:
            case LT.ARRIVAL:
                return acc + 1
            case LT.DEPARTURE:
                return acc - 1

    xs1, xs2 = it.tee((l for l in xs if l.log in [LT.ARRIVAL, LT.DEPARTURE]), 2)

    spans: Iterator[float] = (t2 - t1 for t1, t2 in it.pairwise(mit.prepend(0, (l.timestamp for l in xs1))))
    pkgs: Iterator[int] = it.accumulate((l.log for l in xs2), packages, initial=0)
    return zip(spans, pkgs)


def waiting(xs: Iterator[Log]) -> Iterator[float]:
    """
    From sequence of logs to sequence of waiting times

    >>> list(waiting([Log(10, LT.ARRIVAL), Log(10, LT.SERVING), Log(15, LT.ARRIVAL), Log(20, LT.DEPARTURE), Log(20, LT.SERVING), Log(15, LT.ARRIVAL)]))
    [0, 5]
    """
    xs1, xs2 = it.tee(xs, 2)
    eas = (l.timestamp for l in xs1 if LT.ARRIVAL == l.log)
    ess = (l.timestamp for l in xs2 if LT.SERVING == l.log)
    return (s - a for a, s in zip(eas, ess))


def take(n: int, xs: Iterator[tuple[SupportsFloat, ...]]) -> NDArray[...]:
    """
    From sequence of tuple to NDArray

    >>> take(3, [(1.5, 1), (2.5, 2), (3.5, 3), (4.5, 4)])
    array([[1.5, 1. ],
           [2.5, 2. ],
           [3.5, 3. ]])
    """
    return np.stack([np.asarray(e) for e in zip(*mit.take(n, xs))], axis=1)


def overlapping_batches(xs: Iterator[tuple[SupportsFloat, ...]], b: int, m: int) -> NDArray[...]:
    """
    From sequence of tuple to overlapping batches NDArray

    >>> overlapping_batches([(1.5, 1), (2.5, 2), (3.5, 3), (4.5, 4)], 2, 2)
    array([[[1.5, 1. ],
            [2.5, 2. ]],
    <BLANKLINE>
           [[2.5, 2. ],
            [3.5, 3. ]]])
    """
    vs: NDArray[...] = take(m + b, xs)
    return np.stack([vs[i:(m + i)] for i in range(b)])


def non_overlapping_batches(xs: Iterator[tuple[SupportsFloat, ...]], b: int, m: int) -> NDArray[...]:
    """
    From sequence of tuple to non-overlapping batches NDArray

    >>> non_overlapping_batches([(1.5, 1), (2.5, 2), (3.5, 3), (4.5, 4), (5.5, 5)], 2, 2)
    array([[[1.5, 1. ],
            [2.5, 2. ]],
    <BLANKLINE>
           [[3.5, 3. ],
            [4.5, 4. ]]])
    """
    vs: NDArray[...] = take(m * b, xs)
    return np.stack([vs[i * m:(i + 1) * m] for i in range(b)])


In [ ]:
doctest.testmod()

---

In [ ]:
seed_arr: int = next(seeds)
seed_dep: int = next(seeds)

lmbd: float = 1
mu: float = 10
rho: float = lmbd / mu
gamma: float = 0.95

In [ ]:
span_pkgs: Iterator[tuple[float, int]] = timespan_packages(
    mm1_simulation(seed_arr=seed_arr, seed_dep=seed_dep, lmbd=lmbd, mu=mu)
)

n: int = 500
samples: NDArray[...] = take(n, span_pkgs)

In [ ]:
a: plt.Axes
_, a = plt.subplots(1, 1, figsize=(12, 5))

a.stairs(
    samples[:, 1],
    edges = np.cumsum(np.append(0, samples[:, 0])),
    alpha=0.95,
    label=f'$\\lambda={lmbd}, \\mu={mu}$',
    hatch='\\' * 5,
)

a.axhline(
    rho / (1 - rho),
    alpha=0.5,
    label=r'$\frac{\rho}{1 − \rho}$',
    color='red',
    linestyle='--',
)

a.legend(loc='upper left')
a.set_xlabel('time')
a.set_ylabel('pkgs')
a.set_yticks(range(4))
a.set_title(f'{n} samples, seeds: {seed_arr}, {seed_dep}')
pass

Non-overlapping batches
$$
\underbrace{Y_1, \ldots, Y_m}_{B_1},
\underbrace{Y_{m + 1}, \ldots, Y_{2m}}_{B_2},
\cdots,
\underbrace{Y_{(b - 1) m + 1}, \ldots, Y_{b m}}_{B_b}
$$
$$
\hat V_B = \frac 1 {b - 1} \sum_i^b (Z_i - \overline Z_b)^2
$$
$$
\overline Z_b = \frac 1 b \sum_i^b Z_i
$$
$$
\left[ \overline Z_b \pm t_{b - 1, \frac {1 + \gamma} 2} \sqrt{\frac {\hat V} b} \right]_\gamma
$$

In [ ]:
m: int = 10_000
b: int = 200
n: int = b * m

span_pkgs: Iterator[tuple[float, int]] = timespan_packages(
    mm1_simulation(seed_arr=seed_arr, seed_dep=seed_dep, lmbd=lmbd, mu=mu)
)

batches: NDArray[...] = non_overlapping_batches(span_pkgs, b, m)

In [ ]:
mus: NDArray[float] = np.average(batches[:, :, 1], axis=1, weights=batches[:, :, 0])
grand_mean: float = st.fmean(mus)
v: float = np.sum((mus - grand_mean) ** 2) / (b - 1)

delta: float = sp.stats.t.ppf((1 + gamma) / 2, df=b - 1) * math.sqrt(v / b)

In [ ]:
mus, grand_mean, delta

In [ ]:
a: plt.Axes
_, a = plt.subplots(1, 1, figsize=(12, 5))

a.hlines(
    y=mus,
    xmin=np.arange(0, b) * m,
    xmax=(np.arange(0, b) + 1) * m,
    colors=[f'C{i}' for i in range(b)],
    alpha=0.2,
)

a.axhspan(
    grand_mean - delta,
    grand_mean + delta,
    alpha=0.5,
    color='C0',
    label='CI 0.95'
)
a.axhline(
    grand_mean,
    label=r'$\hat\mu$',
    color='C0',
)
a.axhline(
    rho / (1 - rho),
    alpha=0.5,
    label=r'$\frac{\rho}{1 − \rho}$',
    color='red',
    linestyle='--',
)

a.legend(loc='upper left')
a.set_title(f'non-overlapping {seed_arr}, {seed_dep}')
a.set_xlabel('samples')
a.set_ylabel(r'$\mathbf{E}\left[\frac{\rm pkgs}{\rm timespan}\right]$')
pass

Overlapping batches

$$
\begin{matrix}
Y_1 & Y_2 & Y_3 & Y_4 & \cdots & Y_m \\
 \  & Y_2 & Y_3 & Y_4 & \cdots & Y_m & Y_{m + 1} \\
 \  &  \  & Y_3 & Y_4 & \cdots & Y_m & Y_{m + 1} & Y_{m + 2} \\
 \  &  \  &  \  & Y_4 & \cdots & Y_m & Y_{m + 1} & Y_{m + 2} & Y_{m + 3} \\
 \  &  \  &  \  &  \  & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots
\end{matrix}
$$
$$
\hat V_O = \frac 1 {n - m + 1} \sum_i^{n - m + 1} (Z_i - \overline Z_b)^2
$$
$$
\overline Z_b = \frac 1 b \sum_i^b Z_i
$$
$$
\left[ \overline Z_b \pm t_{b - 1, \frac {1 + \gamma} 2} \sqrt{\frac {\hat V} b} \right]_\gamma
$$

In [ ]:
m: int = 10_000
b: int = 5_000

span_pkgs: Iterator[tuple[float, int]] = timespan_packages(
    mm1_simulation(seed_arr=seed_arr, seed_dep=seed_dep, lmbd=lmbd, mu=mu)
)

batches: NDArray[...] = overlapping_batches(span_pkgs, b, m)

In [ ]:
mus: NDArray[float] = np.average(batches[:, :, 1], axis=1, weights=batches[:, :, 0])
grand_mean: float = st.fmean(mus)
v: float = st.fmean((mus - grand_mean) ** 2)

delta: float = sp.stats.t.ppf((1 + gamma) / 2, df=b - 1) * math.sqrt(v / b)

In [ ]:
mus, grand_mean, delta

In [ ]:
a: plt.Axes
_, a = plt.subplots(1, 1, figsize=(12, 5))

a.hlines(
    y=mus,
    xmin=np.arange(0, b),
    xmax=np.arange(0, b) + 1 + m,
    colors=[f'C{i}' for i in range(b)],
    alpha=0.2,
)

a.axhspan(
    grand_mean - delta,
    grand_mean + delta,
    alpha=0.5,
    color='C0',
    label='CI 0.95'
)
a.axhline(
    grand_mean,
    label=r'$\hat\mu$',
    color='C0',
)
a.axhline(
    rho / (1 - rho),
    alpha=0.5,
    label=r'$\frac{\rho}{1 − \rho}$',
    color='red',
    linestyle='--',
)

a.legend(loc='upper right')
a.set_title(f'overlapping {seed_arr}, {seed_dep}')
a.set_xlabel('samples')
a.set_ylabel(r'$\mathbf{E}\left[\frac{\rm pkgs}{\rm timespan}\right]$')
pass

---

Non-overlapping batches
$$
\underbrace{Y_1, \ldots, Y_m}_{B_1},
\underbrace{Y_{m + 1}, \ldots, Y_{2m}}_{B_2},
\cdots,
\underbrace{Y_{(b - 1) m + 1}, \ldots, Y_{b m}}_{B_b}
$$
$$
\hat V_B = \frac 1 {b - 1} \sum_i^b (Z_i - \overline Z_b)^2
$$
$$
\overline Z_b = \frac 1 b \sum_i^b Z_i
$$
$$
\left[ \overline Z_b \pm t_{b - 1, \frac {1 + \gamma} 2} \sqrt{\frac {\hat V} b} \right]_\gamma
$$

In [ ]:
m: int = 10_000
b: int = 200

ws: Iterator[float] = waiting(
    mm1_simulation(seed_arr=seed_arr, seed_dep=seed_dep, lmbd=lmbd, mu=mu)
)

data: NDArray[float] = np.fromiter(ws, float, n)

batches: NDArray[...] = np.stack([data[i * m:(i + 1) * m] for i in range(b)])

In [ ]:
mus: NDArray[float] = np.average(batches, axis=1)
grand_mean: float = st.fmean(mus)
v: float = np.sum((mus - grand_mean) ** 2) / (b - 1)

delta: float = sp.stats.t.ppf((1 + gamma) / 2, df=b - 1) * math.sqrt(v / b)

In [ ]:
mus, grand_mean, delta

In [ ]:
a: plt.Axes
_, a = plt.subplots(1, 1, figsize=(12, 5))

a.hlines(
    y=mus,
    xmin=np.arange(0, b) * m,
    xmax=(np.arange(0, b) + 1) * m,
    colors=[f'C{i}' for i in range(b)],
    alpha=0.2,
)

a.axhspan(
    grand_mean - delta,
    grand_mean + delta,
    alpha=0.5,
    color='C0',
    label='CI 0.95'
)
a.axhline(
    grand_mean,
    label=r'$\hat\mu$',
    color='C0',
)
a.axhline(
    rho ** 2 / (lmbd * (1 - rho)),
    alpha=0.5,
    label=r'$\frac{\rho^2}{\lambda(1 − \rho)}$',
    color='red',
    linestyle='--',
)

a.legend(loc='upper left')
a.set_title(f'non-overlapping {seed_arr}, {seed_dep}')
a.set_xlabel('samples')
a.set_ylabel(r'$\mathbf{E}\left[{\rm waiting}]\right]$')
pass